In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import urllib.request

In [ ]:
import zipfile
import os

In [ ]:
!wget --no-check-certificate \
    "https://github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/archive/refs/heads/main.zip" \
    -O "/tmp/Arthritis-dataset.zip"



zip_ref = zipfile.ZipFile('/tmp/Arthritis-dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-05-22 00:22:22--  https://github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/zip/refs/heads/main [following]
--2023-05-22 00:22:22--  https://codeload.github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/Arthritis-dataset.zip’

/tmp/Arthritis-data     [              <=>   ]  89.88M  31.5MB/s    in 2.8s    

2023-05-22 00:22:25 (31.5 MB/s) - ‘/tmp/Arthritis-dataset.zip’ saved [94250618]



In [ ]:
# Define dataset paths
base_dir = '/tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
# Define image data generators with data augmentation for training and normalization for validation/testing
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    rotation_range=20,        # Randomly rotate images
    width_shift_range=0.1,    # Randomly shift images horizontally
    height_shift_range=0.1,   # Randomly shift images vertically
    shear_range=0.1,          # Apply shear transformation
    zoom_range=0.1,           # Apply zoom transformation
    horizontal_flip=True,     # Flip images horizontally
    fill_mode='nearest'       # Fill newly created pixels after rotation or shifting
)

In [ ]:
test_val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale pixel values to [0, 1]


In [ ]:
# Define batch size and image size
batch_size = 32
image_size = (224, 224)

In [ ]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 2459 images belonging to 2 classes.


In [ ]:
test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 690 images belonging to 2 classes.


In [ ]:
val_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 355 images belonging to 2 classes.


In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.applications.DenseNet201(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

74836368/74836368 [==============================] - 1s 0us/step


---
**CODE EXPLANATION**

---
The given code defines the architecture of a model using the Keras Sequential API. Here's an explanation of each component:

tf.keras.applications.DenseNet201: This is a pre-trained DenseNet201 model provided by Keras. By setting include_top=False, we exclude the fully connected layers at the top of the network. This allows us to use the pre-trained convolutional base layers for feature extraction.

**weights='imagenet':** This parameter specifies that we want to use the pre-trained weights of the DenseNet201 model that were trained on the ImageNet dataset. These weights capture general image features and can be useful for transfer learning.

**input_shape=(224, 224, 3):** This defines the input shape of the model. In this case, the input shape is set to (224, 224, 3), indicating that the model expects input images of size 224x224 pixels with 3 color channels (RGB).

**tf.keras.layers.GlobalAveragePooling2D():** This layer performs global average pooling on the output of the pre-trained DenseNet201 model. It reduces the spatial dimensions of the feature maps to a vector by taking the average value across each feature map. This helps to extract a compact representation of the features.

**tf.keras.layers.Dense(1, activation='sigmoid'):** This is the final dense layer of the model with 1 neuron. It uses the sigmoid activation function, which maps the output to a range of [0, 1], representing the probability of the input belonging to the positive class. In this case, since it's a binary classification task (healthy or arthritis), we have a single output neuron.

By combining these layers in a sequential manner, we create a model architecture that takes an input image, passes it through the pre-trained DenseNet201 base, performs global average pooling, and finally passes the result through a dense layer with sigmoid activation to obtain the output prediction.

---

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

Epoch 1/5
77/77 [==============================] - 3357s 42s/step - loss: 0.2160 - accuracy: 0.9215 - val_loss: 4.4592 - val_accuracy: 0.9239
Epoch 2/5
77/77 [==============================] - 3175s 41s/step - loss: 0.0730 - accuracy: 0.9768 - val_loss: 0.2213 - val_accuracy: 0.9183
Epoch 3/5
77/77 [==============================] - 3164s 41s/step - loss: 0.0493 - accuracy: 0.9829 - val_loss: 0.4178 - val_accuracy: 0.9408
Epoch 4/5
77/77 [==============================] - 3151s 41s/step - loss: 0.0478 - accuracy: 0.9866 - val_loss: 0.4460 - val_accuracy: 0.9437
Epoch 5/5
28/77 [=========>....................] - ETA: 31:51 - loss: 0.0289 - accuracy: 0.9922

---
**EXPLANATION**

---
The given code is training the model using the fit method of the Keras Model class. Here's what each parameter does:

**train_generator:** It is the generator that yields batches of training data, consisting of input images and their corresponding labels. It is used to train the model on the training dataset.

**epochs:** It specifies the number of times the model will iterate over the entire training dataset during training. In this case, the model will be trained for 10 epochs.

**validation_data:** It is the generator that yields batches of validation data, used to evaluate the model's performance on a separate dataset during training. The validation data is used to monitor the model's progress and prevent overfitting.

During the training process, the model's weights and biases are updated iteratively to minimize the loss function defined in the model's compilation step. The fit method performs forward and backward propagation, calculates gradients, and updates the model parameters using an optimizer. The training process continues for the specified number of epochs, with the model gradually improving its performance on the training and validation data.


---
In the context of training a machine learning model, "epochs" refers to the number of times the model will iterate over the entire training dataset during the training process. Each epoch consists of going through the entire dataset once and updating the model's parameters based on the observed errors.

When you set epochs=10 in the code, it means that the model will undergo 10 complete passes over the training dataset. In each epoch, the model will make predictions on the training data, calculate the loss (error), and adjust its internal parameters (weights and biases) based on the gradient of the loss function. By repeating this process for multiple epochs, the model learns from the data and improves its ability to make accurate predictions.

The number of epochs is a hyperparameter that needs to be carefully chosen. **Too few epochs may result in underfitting, where the model fails to capture the underlying patterns in the data. On the other hand, too many epochs may lead to overfitting, where the model becomes overly specialized to the training data and performs poorly on unseen data. The optimal number of epochs can vary depending on the complexity of the problem and the size of the dataset, and it often requires experimentation and validation on a separate validation set.**

---






In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

22/22 [==============================] - 155s 7s/step - loss: 0.8383 - accuracy: 0.8797


In [ ]:
# Get the true labels for the test set
y_true = test_generator.classes

In [ ]:
# Make predictions on the test set
y_pred_prob = model.predict(test_generator)
y_pred = np.round(y_pred_prob).flatten()

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# Display the confusion matrix
import seaborn as sns

labels = ['Healthy', 'Arthritis']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)

plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

---
**REFERENCES**

---
https://towardsdatascience.com/an-informative-colab-guide-to-load-image-datasets-from-github-kaggle-and-local-machine-75cae89ffa1e

https://pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/